# Multi-Class Classification
이 연습에서는 로지스틱 회귀를 사용하여 손으로 쓴 숫자 (0-9)를 인식합니다. 우리는 실습 2에서 쓴 로지스틱 회귀 분석의 구현을 확장하여 하나 대 모든 분류에 적용 할 것입니다. 데이터 세트를 로드하여 시작합시다. 이것은 MATLAB의 기본 형식이므로 Python으로로드하려면 SciPy 유틸리티를 사용해야합니다.

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('data/ex3data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [6]:
data['X'].shape, data['y'].shape


((5000, 400), (5000, 1))

좋아요, 데이터가로드되었습니다. 이미지는 martix X로 400 차원 벡터로 표현되며 그 중 5,000 개가 벡터이다. 400 개의 "피쳐"는 원본 20 x 20 이미지의 각 픽셀에 대한 그레이 스케일 강도입니다. 클래스 레이블은 이미지에있는 숫자를 나타내는 숫자 클래스로 벡터 y에 있습니다.

MATLAB의 연습 코드에는 손으로 쓴 숫자를 시각화하기 위해 제공되는 함수가 있습니다. 파이썬에서 그 것을 재현하지는 않겠지 만 이미지가 어떻게 보이는지보고 싶다면 연습 PDF에 그림이 있습니다. 로지스틱 회귀 구현으로 넘어갈 것입니다.

첫 번째 작업은 로지스틱 회귀 구현을 완전히 벡터화되도록 수정하는 것입니다 (즉, "for"루프 없음). 이는 벡터화 된 코드가 짧고 간결한 것 외에도 선형 대수 최적화를 이용할 수 있고 일반적으로 반복 코드보다 훨씬 빠르기 때문입니다. 그러나 연습 2에서 비용 함수 구현을 보면 이미 벡터화되어 있습니다! 그래서 우리는 여기서 같은 구현을 재사용 할 수 있습니다. 마지막으로 정규화 된 버전으로 곧바로 건너 뜁니다.

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [8]:
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg